In [ ]:
# we create a function to oobfuscate the store names and introduce impurities for the sake of this excercise
# Scrapped or OCR results could be much more random and unpredictable.
import string
import random 
import numpy as np 
import pandas as pd

seen = {}
symbols = "!@#$%^&*()-+,<.>?/'' "
numbers = [0,1,2,3,4,5,6,7,8,9]


def gen_impurities(n,word):
    for i in range(n):
        index = random.randint(0,len(word)-1)
        #print(f"impute index - {index}")

        if index%2 == 0:
            symbol = random.choice(symbols)
            #print(f"Symbol = {symbol}")
            word = word[:index] + symbol + word[index:]

        if index%7 == 0:
            num = random.choice(numbers)
            #print(f"Number = {num}")
            word = word[:index] + str(num) + word[index:]

        if index%3 == 0:
            word.replace(word[index],"")
    return word

def obfuscate(word):
    n = int(len(word)*0.5)
#     print(n)
    iteration = random.randint(1,n)
#     print(iteration)
    label = word
    for i in range(5):
        imputed_word = gen_impurities(iteration,label)
#         print(imputed_word)
        if imputed_word not in seen:
            seen[imputed_word] = 1
            return imputed_word
        seen[imputed_word] = seen[imputed_word]+1
    return word
            

def gen_gibberish(min_l,max_l):
    # initializing size of string
    l = random.randint(min_l,max_l)
    # using random.choices()
    # generating random strings
    res = ''.join(random.choices(string.ascii_uppercase +
                                 string.punctuation +
                                 string.digits, k=l))
    while res not in seen:    
        seen[res] = 1
        return res
    seen[res] = seen[res]+1
    

In [ ]:
# Lets take top 100 US retailers arbitarily chosen based on their annual reported Sales

def read_data():
    df = pd.read_csv("stores.csv")
    del df['empty']
    df = df.set_index('no')
    df.head()
    return df


In [ ]:
# this function will muddle up the stores to simulate real world data capture where system might introduce impurites
# the function will generate n number of instances for jumbled data

# n indicates number of obfuscated records
# instance_count is the number of observation per class

def get_obfuscated_stores(stores,n):
    obfuscate_stores = stores * n
    obfuscate_stores.sort()
    df_obfuscated = pd.DataFrame({"stores": obfuscate_stores})
    df_obfuscated['bad_names'] = df_obfuscated["stores"].apply(lambda x: obfuscate(x) )
    return df_obfuscated
    
def get_catch_gibberish(instance_count,min_l,max_l):
    others = ['Other'] * instance_count
    df_other = pd.DataFrame({'stores':others})
    df_other['bad_names'] = df_other['stores'].apply(lambda x : gen_gibberish(min_l,max_l)) 
    return df_other

# the training set will also have good captures where store name was interpreted correctly
# for this purpose the we are creagin 10% bad captures

def get_good_names(stores,n):
    good_captures = stores*  n
    df_good_captures = pd.DataFrame({'stores':good_captures})
    df_good_captures['bad_names'] = df_good_captures["stores"]
    return df_good_captures

def get_data(impute_n,instance,min_l,max_l): 
    n = impute_n
    instance_count = instance
    min_l = min_l
    max_l = max_l

    df_obfuscated = get_obfuscated_stores(stores,n)
    df_other = get_catch_gibberish(instance_count,min_l,max_l)
    df_good_captures = repeat_good_names(stores,n,instance_count)
    df = pd.concat([df_obfuscated,df_good_captures,df_other])
    return df


In [ ]:
def get_train_test_data(split_ratio,impute_ratio,instance,min_l,max_l):
    test_size = (instance*split_ratio)
    train_size = (instance*(1-split_ratio))
    print(f"test_size = {test_size} & train_size = {train_size} & split = {split_ratio}")
    test_impute_n = int(test_size*impute_ratio)
    test_good_n = int(test_size - test_impute_n)
    print(f"test impute = {test_impute_n} & instances = {test_good_n}")
    df = read_data()
    stores = list(df["store"])[0:10]
    # get imputed
    df_obfuscated = get_obfuscated_stores(stores,test_impute_n)
    # get good names
    df_good_names = get_good_names(stores, test_good_n)
    # get others 
    df_other = get_catch_gibberish(test_impute_n+test_good_n,min_l,max_l)
    df_test = pd.concat([df_obfuscated,df_good_names,df_other])
    
    del df_obfuscated,df_good_names,df_other
    train_impute_n = int(train_size*impute_ratio)
    train_good_n = int(train_size - train_impute_n)
    # get imputed
    df_obfuscated = get_obfuscated_stores(stores,train_impute_n)
    # get good names
    df_good_names = get_good_names(stores, train_good_n)
    # get others 
    df_other = get_catch_gibberish(train_impute_n+train_good_n,min_l,max_l)
    
    df_train = pd.concat([df_obfuscated,df_good_names,df_other])
    
    del df_obfuscated,df_good_names,df_other
    return df_train, df_test
    
    

In [ ]:
# seen

In [ ]:
df_train, df_test = get_train_test_data(0.2,0.5,10000,5,20)

In [ ]:
print(f"len test stores = {len(set(df_test['stores']))}")
print(f"len train stores = {len(set(df_train['stores']))}")


In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_train[df_train['stores'] == df_train['bad_names']].groupby("stores").count()


In [ ]:
df_test[df_test['stores'] == df_test['bad_names']].groupby("stores").count()


In [ ]:
df_test[df_test["stores"] == 'Other'].count()

In [ ]:
df_train[df_train["stores"] == 'Other'].count()

In [ ]:
df_train.head(10)

In [ ]:
df_test.head(10)

In [ ]:
#from sklearn.model_selection import train_test_split
def get_train_test_split(df_train,df_test,one_hot_encode_labels=False):
    X_train = df_train["bad_names"].values
    y_train = df_train["stores"].values
    X_test  = df_test["bad_names"].values
    y_test  = df_test["stores"].values
    
    if one_hot_encode_labels:
        df_labels = pd.concat([df_train['stores'],df_test['stores']])
        print(f"Labels = {df_labels.shape})")
        lables = pd.get_dummies(df_labels)
        lookup = list(lables.columns)
        print(len(lookup))
        del df_labels
        y_test_labels = pd.get_dummies(y_test)
        y_test_encoded = y_test_labels.astype('float32').values
        y_train_labels = pd.get_dummies(y_train)
        y_train_encoded = y_train_labels.astype('float32').values
        return X_train,y_train_encoded,X_test,y_test_encoded, lookup
    else:
        return X_train, y_train, X_test, y_test, None
    

In [ ]:

X_train, y_train, X_test, y_test, lable_lookup = get_train_test_split(df_train,
                                                                      df_test,
                                                                      one_hot_encode_labels=False)


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
print(len(set(y_train)))
print(len(set(X_train)))

In [ ]:
from sklearn.preprocessing import OneHotEncoder
labels = list(set(y_train))
# enc = OneHotEncoder(handle_unknown = 'ignore')
# enc.fit(np.array(labels).reshape(-1,1))
# y_train_tx = enc.transform(np.array(y_train).reshape(-1,1)).toarray()
# y_test_tx = enc.transform(np.array(y_test).reshape(-1,1)).toarray()

In [ ]:
enc.inverse_transform(test[0].reshape(-1,11))

In [ ]:
print(f"{np.argmax(test[0])} - {y_train[0]} - {labels[0]}")

In [ ]:
def labels_to_index(label,labels=labels):
    return labels.index(label)

In [ ]:
def index_to_labels(idx,labels=labels):
    return labels[idx]

In [ ]:
def get_lables(pred,lookup):
    return lookup[np.argmax(pred)]    

In [ ]:
# one_hot_encode_labels = False
# if one_hot_encode_labels:
#     for i in range(5):
#         print(f" TRAIN : {X_train[i]} >  {get_lables(np.array(y_train[i]),lable_lookup)}")
#     for i in range(5):
#         print(f" TEST : {X_test[i]} >  {get_lables(np.array(y_test[i]),lable_lookup)}")
# else:
#     for i in range(5):
#         print(f" TRAIN : {X_train[i]} >  {y_train[i]}")
#     for i in range(5):
#         print(f" TEST : {X_test[i]} >  {y_test[i]}")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses

In [ ]:
# Vocab = ['UNK']
# Vocab = Vocab+ list(set("".join(X_train)))
# print(len(Vocab))
# #Vocab.append("UNK")
# char_to_ind = {u:i for i, u in enumerate(Vocab)}
# vocab_len = len(Vocab)

In [ ]:
y_train_tx = [labels_to_index(y) for y in y_train]
y_test_tx = [labels_to_index(y) for y in y_test]

In [ ]:
print(y_train_tx[0])
print(y_train[0])
print((y_test_tx)[0])
print(y_test[0])

In [ ]:
string.punctuation

In [ ]:
# ind_to_char = np.array(Vocab)

In [ ]:
# X_train_tx = []
# for x in X_train:
#     X_train_tx.append([char_to_ind[c] for c in x])

In [ ]:
# X_test_tx = []
# for x in X_test:
#     X_test_tx.append([char_to_ind[c] for c in x])

In [ ]:
# y_train_tx = []
# for y in y_train:
#     y_train_tx.append([char_to_ind[c] for c in y])

In [ ]:
# y_test_tx = []
# for y in y_test:
#     y_test_tx.append([char_to_ind[c] for c in y])

In [ ]:
#  def get_word_from_ind(arr):
#     return "".join([ind_to_char[i] for i in arr]).replace("UNK","")

In [ ]:
# print(get_word_from_ind(X_train_tx[0]))
# print(get_word_from_ind(X_test_tx[0]))
# print(get_word_from_ind(y_train_tx[0]))
# print(get_word_from_ind(y_train_tx[0]))

In [ ]:
# x_max_len= 0
# for i in X_train:
#     if len(i)>x_max_len:
#         x_max_len= len(i)
# print(x_max_len)
# x_max_input_length = x_max_len

# x_min_len = np.inf
# for i in X_train:
#     if len(i)<x_min_len:
#         x_min_len = len(i)
# print(x_min_len)
# min_input_length = x_min_len

# y_min_len = np.inf
# for i in y_train:
#     if len(i)<y_min_len:
#         y_min_len = len(i)
# print(y_min_len)

# y_max_len = 0
# for i in y_train:
#     if len(i)>y_max_len:
#         y_max_len = len(i)
# print(y_max_len)


In [ ]:
# input_length =24
# def gen_encoded_strings(word,input_len):
#     word_len = len(word)
#     if word_len > input_len:
#         word = word[0:input_len]
#         return [char_to_ind[c] for c in word]
#     else:
#         encoded = [char_to_ind[c] for c in word]
#         pad_width = input_len-word_len
#         arr = np.array(encoded)
#         return np.pad(arr,(pad_width,0),"constant")
        
    

In [ ]:
# # testing the encoded and decoded.

# s = 'Target'

# s_encoded = gen_encoded_strings(s,input_length)

# print(f" encoded - {s_encoded} - {len(s_encoded)}")

# print(f" decoded - {get_word_from_ind(s_encoded)}")


In [ ]:
# labels = list(set(y_train))

In [ ]:
# def categorical_coding(labels):
#     idx = labels.index(label)
#     return [idx]

# def categorical_label(idx):
#     return labels[idx[0]]
    

In [ ]:
print(y_train[0])
#print(categorical_coding(y_train[0]))
#print(categorical_label(categorical_coding(y_train[0])))

In [ ]:
# max_features = 100
# vectorize_layer = layers.TextVectorization(
#     standardize="lower",
#     split='character',
#     output_sequence_length=max_features,
#     max_tokens=max_features,
#     pad_to_max_tokens=True,
#     output_mode='int')


In [ ]:
# vectorize_layer.adapt(X_train)
# len(vectorize_layer.get_vocabulary())

In [ ]:
# encode both label and test
# def vectorize_text_code_lable(text, label):
#     vector = gen_encoded_strings(text,input_length)
#     idx = categorical_coding(label)
#     return vector, idx

In [ ]:
# vectorize_layer.adapt(
#          X_train
#         )


In [ ]:
# vector_categorical_coding = np.vectorize(categorical_coding)
# vector_gen_encoded_strings = np.vectorize(gen_encoded_strings)

In [ ]:
# X_train_encoded = [gen_encoded_strings(x,input_length) for x in X_train]
# print(f" type - {type(X_train_encoded)}")
# X_test_encoded = [gen_encoded_strings(x,input_length) for x in X_test]
# print(f" type - {type(X_test_encoded)}")


In [ ]:
# y_train_encoded = [gen_encoded_strings(y,input_length) for y in y_train]
# print(f" type - {type(y_train_encoded)}")
# y_test_encoded = [gen_encoded_strings(y,input_length) for y in y_test]
# print(f" type - {type(y_test_encoded)}")


In [ ]:
# y_train_encoded = [categorical_coding(y) for y in y_train]
# type(f"type = {type(y_train_encoded)}")
# y_test_encoded = [categorical_coding(y) for y in y_test]
# type(f"type = {type(y_test_encoded)}")

In [ ]:
# X_train_encoded[0]
# X_test_encoded[0]

In [ ]:
# for x in X_train_encoded:
#     if len(x) != 24:
#         print(len(x))
        
# for y in y_train_encoded:
#     if len(y) != 24:
#         print(len(y))

In [ ]:
# print(type(y_train_encoded[0]))
# print(type(y_test_encoded[0]))


In [ ]:
raw_train = tf.data.Dataset.from_tensor_slices((X_train, y_train_tx))
size = len(raw_train)
raw_train_ds = raw_train.take(int(size*0.8))
raw_val_ds = raw_train.take(int(size*0.2))
raw_test = tf.data.Dataset.from_tensor_slices((X_test, y_test_tx))


In [ ]:
len(raw_train)

In [ ]:
import re
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')


In [ ]:
max_features = 100
sequence_length = 30

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    split = 'character',
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [ ]:
train_text = raw_train.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [ ]:
vectorize_layer.get_vocabulary()

In [ ]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text)[0], label

In [ ]:
text_batch, label_batch = next(iter(raw_train))
#first_review, first_label = text_batch[0], label_batch[0]
print("text", text_batch)
print("Label",label_batch)
print("Vectorized text", vectorize_text(text_batch, label_batch))

In [ ]:
# #lable_lookup
# def map_labels(x,y):
#     print(f"y - {type(y)} - {y.shape}")
#     print(y[0])
#     l = get_lables(y[0],lable_lookup)
#     encoded = gen_encoded_strings(l,input_length)
#     (encoded,y)
#     return x,(encoded,y)
    

In [ ]:

# for example, label in train_data.take(1):
#     print('text: ', example.numpy())
#     print('label: ', label.numpy())
#     x,y = map_labels(example.numpy(),label)
#     #print('encoded label: ', y.shape)
    

In [ ]:
# vectorize_text_code_lable('Target','Target')

In [ ]:
print("3 ---> ",vectorize_layer.get_vocabulary()[3])
print(" 30 ---> ",vectorize_layer.get_vocabulary()[30])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))


In [ ]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test.map(vectorize_text)

In [ ]:
BUFFER_SIZE = 200
BATCH_SIZE = 32

#AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.shuffle(buffer_size=BUFFER_SIZE).batch(batch_size=BATCH_SIZE,drop_remainder=True)
val_ds = val_ds.batch(batch_size=BATCH_SIZE,drop_remainder=True)
test_ds = test_ds.batch(batch_size=BATCH_SIZE,drop_remainder=True)


In [ ]:
for x, y in train_ds.take(1):
    print(x)
    print(y)

In [ ]:
for example, label in train_dataset.take(1):
    print(f"{example.shape} - {label.shape}")
    #print('text: ', example.numpy()[:3])
    #print('label_one_hot: ', label.numpy()[:3])
    #for v in label.numpy()[:3]:
        #print('label ', decode_pred(v,lookup))

In [ ]:
# vocab = np.array(vectorize_layer.get_vocabulary())
# vocab_len = len(vocab)
# print(vocab_len)

In [ ]:
# for x,y in train_dataset.take(1):
#     print(x.shape)
#     print(y.shape)


In [ ]:
# example="Target"
# encoded_example = vectorize_layer(example).numpy()
# encoded_example.shape
len(Vocab)

In [ ]:
max_features

In [ ]:
embedding_dim = 16


def get_model(embedding_dim,max_features):
    model = tf.keras.Sequential([
          layers.Embedding(max_features + 1, embedding_dim),
          layers.Bidirectional(layers.LSTM(128)),
          #layers.Dropout(0.2),
          #layers.GlobalAveragePooling1D(),
          layers.Dropout(0.2),
          layers.Dense(11, activation='softmax')])
    model.summary()
    return model

In [ ]:
model = get_model(embedding_dim,max_features)

In [ ]:
print([layer.supports_masking for layer in model.layers])

In [ ]:
# print(len(vocab))
# sample_text = vectorize_layer('WALMART 1')
# #print(sample_text.shape)
# #print(type(sample_text))
# #print(np.array(sample_text))
# sample = np.array(sample_text).reshape(1,len(vocab))
# print(sample)

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer= 'adam',
              metrics=tf.keras.metrics.SparseCategoricalAccuracy()
             )

In [ ]:
#from tensorflow.keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_loss',patience=5)
# from tensorflow.python.compiler.mlcompute import mlcompute
# tf.compat.v1.disable_eager_execution()
# mlcompute.set_mlc_device(device_name='gpu')
# print("is_apple_mlc_enabled %s" % mlcompute.is_apple_mlc_enabled())
# print("is_tf_compiled_with_apple_mlc %s" % mlcompute.is_tf_compiled_with_apple_mlc())
# print(f"eagerly? {tf.executing_eagerly()}")
print(tf.config.list_logical_devices())

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

In [ ]:
pred_model = get_model(embedding_dim,max_features)

In [ ]:
pred_model.set_weights(model.get_weights())

In [ ]:
pred_model.summary()

In [ ]:
export_model = tf.keras.Sequential([
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=tf.keras.metrics.SparseCategoricalAccuracy(), 
    optimizer="adam",
    metrics=['accuracy']
    )

loss, accuracy , x= export_model.evaluate(test_ds)
print(accuracy)
print(loss)
print(x)

In [ ]:
export_model.summary()

In [1]:
predict_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])


Test_batch = ['WAL1MART','T163TAS']

predict_model.predict(Test_batch)







NameError: name 'tf' is not defined

In [ ]:
predict_model.summary()

In [ ]:
result = predict_model.predict(Test_batch)

In [ ]:
index_to_labels(np.argmax(result))

In [ ]:
def get_predictions_string(text:str):
    
    